In [1]:
# This script is meant to convert the NLDAS data into NetCDF format
# This will be used by the MetSim algorithm to create the radiation/vapor pressure

In [ ]:
"""
Modified on Fri June 5 2020
@author: Jonathan Frame
"""

In [227]:
import datetime as dt
import pandas as pd
import numpy as np
import xarray as xr
import glob
from metsim import MetSim
import netCDF4 as nc
import re

In [2]:
main_dir = '/home/NearingLab/projects/jmframe/CAMELS_synthData/nldas_forcing/'

In [126]:
#Store all attributes. Really just need basin area, but the rest might come in handy.
att_path = "/home/NearingLab/data/camels_attributes_v2.0/camels_all.txt"
attributes = pd.read_csv(att_path, sep=";")
attributes.set_index('gauge_id', inplace=True)

In [127]:
# Get the prefixes for all the forcing data files.
prefix_dict = {'{:02}'.format(pf):[] for pf in range(1, 19)}
for sub_dir_name in glob.glob('/home/NearingLab/projects/jmframe/CAMELS_synthData/nldas_forcing_txt/*'):
    if sub_dir_name == 'readme_git.md':
        continue
    for file_path in glob.glob(sub_dir_name+'/*'):
        prefix = sub_dir_name.split('/')[7]
        temp_basin_id = re.split('_|/', file_path)[11]
        if prefix in prefix_dict.keys():
            prefix_dict[prefix].append(temp_basin_id)
attributes['prefix'] = [np.nan]*attributes.shape[0]
for k in prefix_dict.keys():
    for b in prefix_dict[k]:
        attributes.loc[int(b), 'prefix'] = k

In [306]:
def NLDAS_TXT_TO_NETCDF(basinID):
    basin_id_int = int(basinID)
    basin_id_str = "{:08}".format(basin_id_int)
    latCatchment = attributes.loc[basin_id_int,'gauge_lat']
    lonCatchment = attributes.loc[basin_id_int,'gauge_lon']
    print(latCatchment, lonCatchment)

    # initialize the NetCDF forcing data file.
    data_dir_txt = '/home/NearingLab/projects/jmframe/CAMELS_synthData/nldas_forcing_txt/'
    data_dir_nc = '/home/NearingLab/projects/jmframe/CAMELS_synthData/nldas_forcing_nc/'
    prefix = attributes.loc[basin_id_int, 'prefix']
    path_data_txt = data_dir_txt + prefix + '/' + basin_id_str + '_lump_nldas_forcing_leap.txt'
    data_txt = pd.read_csv(path_data_txt, engine='python', header=3, sep='\t')

    #BEGIN #######################################################################
    #BEGIN #  WRITE DATA TO THE NETCDF FORCING DATA FILE   #######################
    #BEGIN #######################################################################

    forcingDataName = data_dir_nc + basin_id_str + '_forcing.nc'
    forcing = nc.Dataset(forcingDataName, 'w', format='NETCDF4_CLASSIC')
    forcing.title = basin_id_str +' _forcing'
    forcing.description = 'Forcing data for basin '+ basin_id_str + ' in the CAMELS catchments'

    forcing.createDimension('lat', 1)
    forcing.createDimension('lon', 1)
    forcing.createDimension('time', size=data_txt.shape[0])

    ### createVareables in new data set
    forcing.createVariable('Prec', np.int32, ('time',))
    forcing.variables['Prec'].units = 'mm'
    forcing.variables['Prec'].long_name = 'Precipitation'
    #forcing.variables['Prec'].v_type = 'scalarv'
    forcing.variables['Prec'][:] = np.array(data_txt['PRCP(mm/day)'])

    forcing.createVariable('Tmax', np.float32, ('time',))
    forcing.variables['Tmax'].units = 'C' 
    forcing.variables['Tmax'].longname = "Daily maximum temperature"
    #forcing.variables['Tmax'].v_type = 'scalarv'
    forcing.variables['Tmax'][:] = np.array(data_txt['Tmax(C)'])
    
    forcing.createVariable('Tmin', np.float32, ('time',))
    forcing.variables['Tmin'].units = 'C'
    forcing.variables['Tmin'].longname = "Daily minimum temperature"
    #forcing.variables['Tmin'].v_type = 'scalarv'
    forcing.variables['Tmin'][:] = np.array(data_txt['Tmin(C)'])
    
    forcing.createVariable('latitude', np.float32, ('lat',))
    forcing.variables['latitude'].units = 'degrees_north'
    forcing.variables['latitude'].long_name = 'latitude'
    forcing.variables['latitude'].axis = 'Y'
    forcing.variables['latitude'][:] = latCatchment

    forcing.createVariable('longitude', np.float32, ('lon',))
    forcing.variables['longitude'].units = 'degrees-east'
    forcing.variables['longitude'].long_name = 'longitude'
    forcing.variables['longitude'].axis = 'X'
    forcing.variables['longitude'][:] = lonCatchment

    forcing.createVariable('time', np.float64, ('time',))
    forcing.variables['time'].units = 'hours since 2000-01-01 00:00:00'
    forcing.variables['time'].long_name = "local time at grid location"
    forcing.variables['time'].calendar = 'standard'
    time_pd = pd.date_range(start='1/1/1980', end='12/31/2014', freq='1D')
    nldas_time = [dt.datetime(1980, 1, 1) + dt.timedelta(days=day) 
                  for day in range(data_txt.shape[0])]
    forcing.variables['time'][:] = nc.date2num(nldas_time, 
                                          units='hours since 2000-01-01 00:00', calendar='standard')
    
    forcing.close()

In [307]:
%%bash
mv nldas_forcing_nc/01013500_forcing.nc z_trash/

In [308]:
for b in attributes.index.values:
    NLDAS_TXT_TO_NETCDF(b)

47.23739000000001 -68.58264
[0 0 0 ... 0 0 0]
[ -7.8   -6.2  -11.26 ...  -0.77 -12.76 -17.05]
[-17.66 -14.01 -16.3  ... -12.1  -18.06 -23.4 ]
[47.23739]
[-68.58264]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
44.60797 -67.93524000000001
[0 0 0 ... 0 0 0]
[-2.84 -0.62 -2.3  ...  2.96 -2.64 -7.39]
[ -8.69  -4.97  -9.11 ...  -2.16  -9.63 -12.17]
[44.60797]
[-67.93524]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
45.50097 -68.30596
[0 0 0 ... 0 0 0]
[ -5.28  -3.37  -5.37 ...   1.1   -6.73 -12.06]
[-11.24  -7.89 -11.88 ...  -6.18 -13.09 -17.05]
[45.50097]
[-68.30596]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
45.17501 -69.3147
[0 0 0 ... 0 0 0]
[ -4.22  -2.99  -5.32 ...   0.05  -6.06 -10.64]
[-11.19  -7.79 -11.5  ...  -5.71 -12.56 -15.94]
[45.17501]
[-69.3147]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
44.8692 -69.9551
[0 0 0 ... 0 0 0]
[ -4.3   -3.03  -6.03 ...  -0.56  -6.78 -10.81]
[-11.29  -8.24 -12.13 ...  -6.42 -12.86 -15

[0 0 0 ... 0 0 0]
[ 3.76  4.34  0.08 ...  5.91  0.95 -3.17]
[-3.36 -1.91 -2.53 ... -1.01 -4.02 -4.76]
[41.10667]
[-74.95222]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
41.08481 -75.21462
[0 0 0 ... 0 0 0]
[ 2.88  3.04 -1.03 ...  4.63 -0.03 -4.33]
[-4.65 -3.48 -3.46 ... -2.26 -4.7  -5.65]
[41.08481]
[-75.21462]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
40.66176 -75.62685
[0 0 0 ... 0 0 0]
[ 4.2   4.51  1.23 ...  7.29  2.05 -2.48]
[-3.13 -2.08 -1.63 ...  1.32 -3.09 -4.12]
[40.66176]
[-75.62685]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
39.885 -74.50528
[0 0 0 ... 0 0 0]
[5.99 6.46 2.89 ... 8.49 4.01 0.36]
[-1.79 -0.65 -1.96 ...  2.95 -2.03 -4.  ]
[39.885]
[-74.50528]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
38.90578 -75.51275
[0 0 0 ... 1 0 0]
[6.32 6.06 3.68 ... 9.   5.93 1.46]
[ 0.01 -0.13  0.19 ...  5.88  0.13 -2.39]
[38.90578]
[-75.51275]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
38.22892 -75.47

[0 0 0 ... 0 0 0]
[ 4.17  3.42  2.49 ...  8.1   3.68 -1.48]
[-2.91 -3.43 -3.3  ...  2.26 -1.53 -5.25]
[39.08122]
[-78.32945]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
39.25455 -77.57638
[0 0 0 ... 0 0 0]
[ 4.97  4.61  3.   ...  8.48  4.61 -0.23]
[-3.19 -2.88 -2.89 ...  3.07 -0.7  -4.72]
[39.25455]
[-77.57638]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
39.61236 -77.23744
[0 0 0 ... 0 0 0]
[ 4.35  4.07  1.82 ...  8.17  3.88 -1.91]
[-3.69 -3.12 -1.9  ...  1.4  -1.81 -4.91]
[39.61236]
[-77.23744]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
39.01955 -77.57749
[0 0 0 ... 0 0 0]
[5.21 4.81 3.37 ... 8.76 4.65 0.17]
[-2.73 -2.55 -2.57 ...  3.78 -0.55 -4.24]
[39.01955]
[-77.57749]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
38.58734000000001 -77.4286
[0 0 0 ... 1 0 0]
[6.12 6.08 4.02 ... 8.88 5.34 1.8 ]
[-2.57 -2.98 -2.21 ...  5.47  0.05 -4.15]
[38.58734]
[-77.4286]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
38.5

[ 0  0  0 ... 21  0  0]
[ 9.03  9.95  9.98 ... 15.62 10.47  7.42]
[ 5.49 -0.6   0.5  ... 11.09  4.42 -1.63]
[35.18278]
[-79.1775]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
34.82889 -77.83221999999998
[0 0 0 ... 5 0 0]
[10.2   9.88  9.51 ... 16.64 11.27  7.31]
[ 6.11 -0.62 -0.2  ... 12.01  5.05 -1.9 ]
[34.82889]
[-77.83222]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
36.07139 -81.40306
[ 0  0  0 ... 14  6  0]
[ 4.46  4.99  5.63 ... 10.25  7.73  1.53]
[ 1.28 -2.75 -0.69 ...  7.98  2.13 -1.91]
[36.07139]
[-81.40306]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
36.175 -81.16889
[ 0  0  0 ... 15  8  0]
[ 5.27  5.87  6.24 ... 11.09  7.95  2.27]
[ 1.4  -2.48 -0.22 ...  8.08  2.62 -1.75]
[36.175]
[-81.16889]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
36.25028 -81.04444000000002
[ 0  0  0 ... 14  9  0]
[ 5.1   5.94  6.04 ... 10.94  7.4   2.29]
[ 0.64 -2.8  -0.57 ...  7.67  2.23 -2.38]
[36.25028]
[-81.04444]
[-175320. -175296. -17

[ 0  0  0 ... 13  0  0]
[ 9.59 13.54 16.26 ... 24.68 22.05 19.73]
[ 3.82  0.55  2.57 ... 15.12 16.17 11.41]
[30.09827]
[-83.47181]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
30.12937 -84.49435
[0 0 0 ... 8 0 0]
[ 9.5  13.88 16.69 ... 22.37 20.79 19.56]
[ 4.38  1.71  4.29 ... 15.21 14.8   8.24]
[30.12937]
[-84.49435]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
32.01766 -85.29576999999998
[0 0 0 ... 4 0 0]
[ 8.41 12.08 13.84 ... 20.61 17.5  15.59]
[ 4.85  0.    1.06 ... 15.57  9.36  4.26]
[32.01766]
[-85.29577]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
32.19556 -83.90222
[ 0  0  0 ... 14  0  0]
[ 8.8  12.41 14.16 ... 20.96 18.38 14.8 ]
[ 5.53 -1.57  0.21 ... 15.34 10.52  6.36]
[32.19556]
[-83.90222]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
31.764440000000004 -84.25333
[ 0  0  0 ... 14  0  0]
[ 7.93 12.07 14.18 ... 20.74 18.1  15.13]
[ 4.97 -0.66  2.   ... 15.62 10.22  6.85]
[31.76444]
[-84.25333]
[-175320. -175296. -17

...

[0 0 0 ... 0 0 1]
[17.58 19.55 16.2  ... 16.35 15.61 13.28]
[6.24 7.77 6.86 ... 2.85 5.65 3.46]
[29.37079]
[-99.28782]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
36.54169 -105.5564
[0 0 0 ... 0 0 1]
[ -0.73  -4.51  -5.72 ... -10.03 -10.76  -5.36]
[ -6.67  -9.76 -13.19 ... -14.37 -18.59 -15.52]
[36.54169]
[-105.5564]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
36.43947 -105.50362
[0 0 0 ... 0 1 0]
[ 0.41 -2.93 -4.2  ... -8.4  -9.03 -4.41]
[ -5.5   -8.25 -11.55 ... -13.58 -16.6  -13.97]
[36.43947]
[-105.50362]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
36.50836 -105.53084
[0 0 0 ... 0 0 1]
[ -1.21  -4.99  -6.15 ... -10.49 -11.24  -5.77]
[ -7.1  -10.23 -13.7  ... -14.81 -19.15 -16.07]
[36.50836]
[-105.53084]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
35.66169 -106.74337
[0 0 0 ... 0 0 0]
[ 1.87 -1.61 -2.6  ... -3.96 -7.06 -5.13]
[ -4.7   -7.03  -9.71 ... -10.75 -12.62 -12.8 ]
[35.66169]
[-106.74337]
[-175320. -175296. -175

[0 0 0 ... 0 0 3]
[13.06 11.71 10.47 ...  8.81  8.52  5.04]
[ 4.01  1.84 -0.98 ... -3.32 -3.39 -1.59]
[34.16087]
[-111.69292]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
33.69421 -111.5418
[0 0 0 ... 0 0 9]
[16.74 15.14 13.92 ... 10.49 10.62  7.74]
[ 5.43  3.79  1.32 ... -1.21 -0.99  0.92]
[33.69421]
[-111.5418]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
33.88726 -111.95404
[0 0 0 ... 0 0 6]
[14.5  13.59 12.44 ... 10.23  9.94  7.75]
[ 4.64  3.45  0.93 ... -1.29 -1.22  1.41]
[33.88726]
[-111.95404]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
33.9742 -112.09905
[0 0 0 ... 0 0 5]
[14.12 13.35 12.3  ... 10.24  9.93  7.8 ]
[ 4.74  3.5   1.07 ... -1.02 -1.17  1.34]
[33.9742]
[-112.09905]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
41.60994 -111.25409
[0 0 0 ... 0 0 0]
[ -0.38  -2.2   -1.7  ...  -8.36 -12.39 -13.65]
[ -3.45  -5.18  -6.99 ... -11.41 -22.5  -25.77]
[41.60994]
[-111.25409]
[-175320. -175296. -175272. ...  131424.  

[4 0 0 ... 0 0 0]
[ 2.69  7.21  8.03 ... -1.83 -3.59 -6.29]
[ -2.38  -2.72  -1.95 ...  -8.46 -12.44 -13.71]
[37.19856]
[-119.21373]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
36.40217 -120.43349
[0 0 0 ... 0 0 0]
[15.49 14.53 15.75 ...  9.98  9.73  6.96]
[ 8.66  5.13  5.76 ...  1.43  0.8  -0.77]
[36.40217]
[-120.43349]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
37.73159 -119.55877
[6 0 0 ... 0 0 0]
[ 0.19  6.17  6.01 ... -1.36 -3.23 -8.37]
[ -5.51  -3.5   -2.93 ...  -7.73 -14.57 -15.89]
[37.73159]
[-119.55877]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
37.71687 -119.66628
[8 0 0 ... 0 0 0]
[ 1.1   7.05  6.96 ... -0.39 -2.22 -7.34]
[ -4.46  -2.6   -1.93 ...  -6.88 -13.21 -14.96]
[37.71687]
[-119.66628]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
37.31549 -121.12521
[0 0 0 ... 0 0 0]
[15.85 14.86 15.51 ... 11.62 11.28  8.41]
[8.73 4.86 5.99 ... 2.39 2.33 0.09]
[37.31549]
[-121.12521]
[-175320. -175296. -175272. ...  13142

[16  3  0 ...  1  0  0]
[ 3.15  3.01  2.26 ...  0.24 -4.89 -1.36]
[  1.89   0.22  -4.64 ...  -4.66 -11.1  -10.67]
[47.35066]
[-121.66316]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
47.38649 -121.84622
[15  4  1 ...  2  0  0]
[ 5.17  5.3   4.4  ...  2.64 -2.09 -0.53]
[ 3.45  1.61 -1.04 ... -1.5  -8.19 -7.55]
[47.38649]
[-121.84622]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
47.48594 -121.64788
[21  7  2 ...  3  1  0]
[ 2.51  2.47  1.02 ... -1.07 -5.92 -1.72]
[  0.93  -0.16  -3.88 ...  -6.1  -12.35 -10.78]
[47.48594]
[-121.64788]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
47.4526 -121.71788
[20  5  1 ...  2  2  0]
[ 2.85  2.74  1.61 ... -0.46 -5.55 -1.44]
[  1.48   0.2   -4.45 ...  -5.49 -11.83 -11.05]
[47.4526]
[-121.71788]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
47.49288 -121.79011
[19  5  1 ...  2  1  0]
[ 3.49  3.34  2.3  ...  0.31 -4.68 -1.11]
[  2.01   0.66  -3.49 ...  -4.52 -10.91 -10.13]
[47.49288]
[-121.79011

[5 0 6 ... 2 0 0]
[ 4.48  4.07  2.02 ... -3.74 -8.34 -7.51]
[  1.29  -1.98  -0.51 ...  -7.59 -18.68 -17.21]
[46.08639]
[-115.97667]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
46.47833 -116.2575
[7 1 7 ... 2 0 0]
[  3.85   3.13   1.39 ...  -4.75 -10.27  -8.17]
[  0.89  -2.45  -1.28 ...  -9.58 -19.97 -17.66]
[46.47833]
[-116.2575]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
46.84047 -115.6207
[12  4  8 ...  0  0  0]
[  2.86   2.18   0.88 ...  -5.47 -12.16  -8.92]
[  1.07  -3.1   -3.54 ... -12.86 -23.13 -18.62]
[46.84047]
[-115.6207]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
45.71958 -118.32329
[4 0 8 ... 7 0 0]
[ 6.12  4.77  3.63 ... -1.09 -8.   -8.47]
[  3.11   0.16   0.17 ...  -7.25 -17.11 -17.64]
[45.71958]
[-118.32329]
[-175320. -175296. -175272. ...  131424.  131448.  131472.]
44.75039 -121.6334
[ 6  1  2 ... 10  0  0]
[ 3.01  3.09  2.63 ... -2.29 -8.16 -4.29]
[ -0.13  -0.97  -2.01 ...  -7.36 -13.84 -12.96]
[44.75039]
[-121.6334]
[-